# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [117]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [118]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 65 columns):
id                    4870 non-null object
ad                    4870 non-null object
education             4870 non-null int8
ofjobs                4870 non-null int8
yearsexp              4870 non-null int8
honors                4870 non-null int8
volunteer             4870 non-null int8
military              4870 non-null int8
empholes              4870 non-null int8
occupspecific         4870 non-null int16
occupbroad            4870 non-null int8
workinschool          4870 non-null int8
email                 4870 non-null int8
computerskills        4870 non-null int8
specialskills         4870 non-null int8
firstname             4870 non-null object
sex                   4870 non-null object
race                  4870 non-null object
h                     4870 non-null float32
l                     4870 non-null float32
call                  4870 non-null float32
city        

### 1. What test is appropriate for this problem? Does CLT apply?

#### 1. Solution

The appropriate test is a two sample proportion z test. CLT does apply, because sample size is greater than 30. 

### 2. What are the null and alternate hypotheses?

In [8]:
# Null Hypothesis: the difference in proportion of call backs between whites and blacks equals 0
# Alternative Hypothesis: the difference in proportion of call backs between whites and blacks does not equal 0

### 3. Compute margin of error, confidence interval, and p-value.

In [9]:
successB = sum(data[data.race=='b'].call)
successW = sum(data[data.race=='w'].call)
dataBlack = data[data['race'] == 'b']
dataWhite = data[data['race'] == 'w']
lenB = len(dataBlack)
lenW = len(dataWhite)
successRatioW = successW / lenW
successRatioB = successB / lenB
print('success ratio white: ', successRatioW, 'success ratio black: ', successRatioB)

success ratio white:  0.0965092402464 success ratio black:  0.064476386037


In [10]:
pointEst = successRatioW - successRatioB
pointEst

0.032032854209445585

In [11]:
std = np.sqrt(((successRatioW*(1 - successRatioW))/lenW + ((successRatioB*(1 - successRatioB))/lenB)))
print(std)

0.00778337058668


In [12]:
# When the observed number of successes and the observed number of 
# failures are greater than or equal to 5 for both populations, 
# then the sampling distribution of  p̂ 1−p̂ 2 is approximately normal 
# and we can use z-methods.

In [13]:
z = 1.96
me = z * std 
lower = pointEst + me
upper = pointEst - me
prop = (successW + successB) / (lenW + lenB)
Zstat = pointEst / np.sqrt((prop*(1-prop)*((1/lenW)+(1/lenB))))
# p = 2 * probability(z<-4.1084)
# p = 2 * 0.00002
p = 2 * 0.00002
print('Margin of error is: ', me)
print('Confidence interval is: ' , upper, " - ", lower)
print('Z is: ', z)
print('Z-stat is: ', Zstat)
print('P-value is: ', p)

Margin of error is:  0.0152554063499
Confidence interval is:  0.0167774478596  -  0.0472882605593
Z is:  1.96
Z-stat is:  4.10841215243
P-value is:  4e-05


In [15]:
import numpy as np
from statsmodels.stats.proportion import proportions_ztest
counts = np.array([successB, successW])
nobs = np.array([lenB, lenW])
stat, pval = proportions_ztest(counts, nobs)
print('{0:0.3f}'.format(stat))
print('{0:0.3f}'.format(pval))

-4.108
0.000


### 4. Write a story describing the statistical significance in the context or the original problem.

The Z-statisitic of 4.1084 is greater than 1.96, thus we rejuect the Null Hypothesis and accept that at a .05 significane level the difference between call back proportions of whites vs blacks is not zero

4e-05 is less than our confidence level of .05, thus we can reject our null hypothsis and assume there is a difference in proportions

### 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

This analysis only looks at the influence of race/name on callback success. It is out of the scope of this analysis to compare importance of factors on callback success, given it is only evaluating a single factor. In order to determine the importance of factors, the study could be amended by considering the impact and statisitical significance of other factors.